# Data Acquisition

### Import Dependencies

In [61]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd
from datetime import datetime

In [4]:
def get_jobs(keyword, num_jobs, verbose, path, slp_time):
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''

    # Initializing the webdriver
    options = webdriver.ChromeOptions()

    # Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    # options.add_argument('headless')

    # Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    
    url = 'https://www.glassdoor.co.uk/Job/uk-' + keyword + '-jobs-SRCH_IL.0,2_IN2_KO3,17.htm'
#     url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  # If true, should be still looking for new jobs.

        # Let the page load. Change this number based on your internet speed.
        # Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slp_time)

        # Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_xpath('.//li[@data-selected="true"]').click()
        except ElementClickInterceptedException:
            pass

        time.sleep(0.1)

        try:
            driver.find_element_by_class_name("modal_closeIcon").click() # clicking to the X.
        except NoSuchElementException:
            pass
        
        #Removing the cookies banner
        try:
            driver.find_element_by_id("onetrust-accept-btn-handler").click()
        except:
            pass
        
        # Going through each job in this page
        job_buttons = driver.find_elements_by_class_name(
            "react-job-listing")  # jl for Job Listing. These are the buttons we're going to click.

        for job_button in job_buttons:

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click() 
            time.sleep(1)
            collected_successfully = False

            while not collected_successfully:
                try:
                    location = job_button.get_attribute("data-job-loc")
                    job_title = job_button.get_attribute("data-normalize-job-title")
                    job_description = driver.find_element_by_xpath('//*[@id="JobDescriptionContainer"]').text
                    company_name = driver.find_element_by_css_selector(".css-87uc0g.e1tk4kwz1").text
                    collected_successfully = True
                    
                except Exception as e:
                    print(e)
                    time.sleep(5)
        
            try:
                salary_estimate = driver.find_element_by_xpath('//*[@id="JDCol"]/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[4]/span').text
    
            except NoSuchElementException:
                salary_estimate = '-1'  # You need to set a "not found value. It's important."

            try:
#                 rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
                
                rating = driver.find_element_by_css_selector("span[data-test='detailRating']").text
            except NoSuchElementException:
                rating = -1  # You need to set a "not found value. It's important."

            # Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            try:

                try:
                    size = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[1]/span[2]').text
                    
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[2]/span[2]').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[3]/span[2]').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:        
                    industry = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[4]/span[2]').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[5]/span[2]').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('//*[@id="EmpBasicInfo"]/div[1]/div/div[6]/span[2]').text
                except NoSuchElementException:
                    revenue = -1

            except NoSuchElementException:  # Rarely, some job postings do not have the "Company" tab.
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1

            if verbose:
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title": job_title,
                         "Salary Estimate": salary_estimate,
                         "Job Description": job_description,
                         "Rating": rating,
                         "Company Name": company_name,
                         "Location": location,
                         "Size": size,
                         "Founded": founded,
                         "Type of ownership": type_of_ownership,
                         "Industry": industry,
                         "Sector": sector,
                         "Revenue": revenue
                         })
            # add job to jobs

        # Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs,
                                                                                                         len(jobs)))
            break

    return pd.DataFrame(jobs)  # This line converts the dictionary object into a pandas DataFrame.

In [5]:
path = "../chromedriver/chromedriver.exe"

df = get_jobs('data-scientist', 10, False, path, 7)

df.head(10)

Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10
Scraping terminated before reaching target number of jobs. Needed 10, got 10.


In [8]:
df.to_csv("glassdoor_jobs.csv", index=False)

# Data Cleaning

### Salary Parsing



In [53]:
# Make new columns for hourly or employed provided salaries
df = pd.read_csv("glassdoor_jobs.csv")

df['Hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df['Employer Provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

# Remove rows without salary records
df= df[df['Salary Estimate'] != "-1"]

# Remove any text and pound sign symbols from the cells
salary = df["Salary Estimate"].apply(lambda x: x.split('(')[0])
minus_kd = salary.apply(lambda x: x.replace('K', "").replace("£",""))

min_hr = minus_kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary',''))

df['Min Salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))
df['Max Salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))
df['Average Salary'] = (df['Max Salary'] + df['Min Salary'])/2
df

,Company Name,Founded,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,Size,Type of ownership,Hourly,Employer Provided,Min Salary,Max Salary,Average Salary
0,CGI\n3.7,1976,Consulting,Job Description Senior Data Scientist\n\nPosit...,Senior Data Scientist,"Reading, England",3.7,$10+ billion (USD),£50K - £65K (Glassdoor Est.),Business Service,10000+ Employees,Company - Public,0,0,50,65,57.5
1,Sage\n4.3,1981,Computer Hardware & Software,Advert\nPeople make Sage great. From our colle...,Senior Data Scientist,"London, England",4.3,$1 to $2 billion (USD),£51K - £61K (Glassdoor Est.),Information Technology,10000+ Employees,Company - Public,0,0,51,61,56.0
3,Yara\n4.1,1905,Chemical Manufacturing,About the Unit\nYara delivers solutions for su...,Research Scientist,"Leeds, England",4.1,$10+ billion (USD),£25K - £35K (Glassdoor Est.),Manufacturing,10000+ Employees,Company - Public,0,0,25,35,30.0
4,PsiOxus Therapeutics Ltd,-1,-1,PsiOxus Therapeutics Ltd is an award-winning U...,Senior Research Scientist,"Abingdon, England",-1.0,-1,£42K - £65K (Glassdoor Est.),-1,-1,-1,0,0,42,65,53.5
5,FutureLearn Ltd\n3.4,2012,Education Training Service,We are currently looking to recruit an experie...,Data Scientist,"London, England",3.4,Unknown / Non-Applicable,£34K - £40K (Glassdoor Est.),Education,51 to 200 Employees,Company - Private,0,0,34,40,37.0
7,Funding Circle UK\n3.9,2010,Lending,About Us\nWe want to help small businesses win...,Data Scientist,"London, England",3.9,$50 to $100 million (USD),£51K - £66K (Glassdoor Est.),Finance,501 to 1000 Employees,Company - Private,0,0,51,66,58.5
8,Conrad Energy,-1,-1,Quantitative Analyst/Data Scientist\nThe quant...,Quantitative Analyst,"Abingdon, England",-1.0,-1,£38K - £72K (Glassdoor Est.),-1,-1,-1,0,0,38,72,55.0
9,QVC\n3.5,1986,"Department, Clothing, & Shoe Shops",QVC is so much more than a retailer. It’s a br...,Data Engineer,"London, England",3.5,$5 to $10 billion (USD),£43K - £58K (Glassdoor Est.),Retail,10000+ Employees,Subsidiary or Business Segment,0,0,43,58,50.5


### Parse Other Data

In [78]:

# Company name with text only 
df['Company Name'] = df['Company Name'].apply(lambda x: x.splitlines()[0])
                                          
# Country field
df['Country'] = df['Location'].apply(lambda x: x.split(',')[1])

# Age of company
df['Company Age'] = df['Founded'].apply(lambda x: x if x < 0 else (datetime.now().year - x))

# Parsing of job description (python, SQL, AWS, Spark, Excel, etc)
df['Python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['SQL'] = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
df['AWS'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
df['Spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df['Excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df.to_csv('salary_data_cleaned.csv')
df


,Company Name,Founded,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,...,Min Salary,Max Salary,Average Salary,Country,Company Age,Python,SQL,AWS,Spark,Excel
0,CGI,1976,Consulting,Job Description Senior Data Scientist\n\nPosit...,Senior Data Scientist,"Reading, England",3.7,$10+ billion (USD),£50K - £65K (Glassdoor Est.),Business Service,...,50,65,57.5,England,45,0,0,0,0,0
1,Sage,1981,Computer Hardware & Software,Advert\nPeople make Sage great. From our colle...,Senior Data Scientist,"London, England",4.3,$1 to $2 billion (USD),£51K - £61K (Glassdoor Est.),Information Technology,...,51,61,56.0,England,40,0,0,0,0,0
3,Yara,1905,Chemical Manufacturing,About the Unit\nYara delivers solutions for su...,Research Scientist,"Leeds, England",4.1,$10+ billion (USD),£25K - £35K (Glassdoor Est.),Manufacturing,...,25,35,30.0,England,116,0,0,0,0,0
4,PsiOxus Therapeutics Ltd,-1,-1,PsiOxus Therapeutics Ltd is an award-winning U...,Senior Research Scientist,"Abingdon, England",-1.0,-1,£42K - £65K (Glassdoor Est.),-1,...,42,65,53.5,England,-1,0,0,0,0,0
5,FutureLearn Ltd,2012,Education Training Service,We are currently looking to recruit an experie...,Data Scientist,"London, England",3.4,Unknown / Non-Applicable,£34K - £40K (Glassdoor Est.),Education,...,34,40,37.0,England,9,1,0,1,1,0
7,Funding Circle UK,2010,Lending,About Us\nWe want to help small businesses win...,Data Scientist,"London, England",3.9,$50 to $100 million (USD),£51K - £66K (Glassdoor Est.),Finance,...,51,66,58.5,England,11,0,0,0,0,0
8,Conrad Energy,-1,-1,Quantitative Analyst/Data Scientist\nThe quant...,Quantitative Analyst,"Abingdon, England",-1.0,-1,£38K - £72K (Glassdoor Est.),-1,...,38,72,55.0,England,-1,0,0,0,0,0
9,QVC,1986,"Department, Clothing, & Shoe Shops",QVC is so much more than a retailer. It’s a br...,Data Engineer,"London, England",3.5,$5 to $10 billion (USD),£43K - £58K (Glassdoor Est.),Retail,...,43,58,50.5,England,35,0,0,0,0,0
